# Notebook to extract the various nested data within a CSV file via a "strong-arm" methodology for later use in a Kaggle Competition building Recommender Systems.

In [1]:
import pandas as pd
import numpy as np
import ast
import time
from tqdm.notebook import tqdm_notebook as tqdm

## Going to start by expanding the Metadata CSV file.

In [2]:
def retrieve_nested_data_metadata():
    '''Function to Retrieve the nested data in the Metadata file'''
    
    df = pd.read_csv("movies_metadata.csv")
    
    # Creating column names to account for the amount of sub-categories within each entry
    col_names = ['genre_id_1', 'genre_name_1',
                 'genre_id_2', 'genre_name_2', 
                 'genre_id_3', 'genre_name_3', 
                 'genre_id_4', 'genre_name_4', 
                 'genre_id_5', 'genre_name_5', 
                 'genre_id_6', 'genre_name_6', 
                 'genre_id_7', 'genre_name_7',
                 'genre_id_8', 'genre_name_8']
    
    # Adding those columns to the dataframe
    df = df.reindex(columns = df.columns.tolist() + col_names)
    
    # Iterating through the dataframe with the iterrows method (its kinda like enumerate for DF's), 
    # using tqdm as a timer. Index from the "enumeration" is helpful for multi-indexing later.
    for index, row in tqdm(df.iterrows(), desc = 'Genres'):
        
        # Using ast library to evaluate a string formatted as a dictionary, which basically typecasts. 
        ## ast library works better than python's eval method IMO.
        genre_data = ast.literal_eval(row[3])
        
        # Using enumerate here to be able to perform efficient multi-indexing.
        for counter, dictionary in enumerate(genre_data, start = 1):
            
            # By using a dict based for loop, and if/else block makes this multi-indexing easier, and allows
            # for us to easily assign values to DF rows based on our conditional. NOTE: df.loc is the best
            # way to multi-index through a DF that I have found. (had errors trying to use list-like multi-indexing,
            # (ie) df['col']['index'])
            for key, value in dictionary.items():
                if key == 'id':
                    df.loc[index, f'genre_id_{counter}'] = value

                elif key == 'name':
                    df.loc[index, f'genre_name_{counter}'] = value

    # Creating column names to account for the amount of sub-categories within each entry
    col_names = ['collection_id', 'part_of_collection']
    
    # Adding those columns to the dataframe
    df = df.reindex(columns = df.columns.tolist() + col_names)
    
    # Iterating through the dataframe with the iterrows method, using tqdm as a timer. Don't need to multi-index this time.
    for index, row in tqdm(df.iterrows(), desc = 'Belongs to Collection'):
        
        # Using a try/ except suite since ast's eval methods throw an error for the various NaN entries in the column.
        try:
            
            # Using ast library to evaluate a string formatted as a dictionary, which basically typecasts. 
            ## ast library works better than python's eval method IMO.
            collection_data = ast.literal_eval(row[1])
            
            # Same idea here with the key, value for loop, making it easier to assign specific cells.
            for key, value in collection_data.items():
   
                if key == 'id':
                    df.loc[index, 'collection_id'] = value

                elif key == 'name':
                    df.loc[index, 'part_of_collection'] = value
        
        # Here the pass statement is just making it so that we skip over each row with NaN values
        except ValueError:
            pass
    
    # Returning our expanded dataframe!
    return df

In [6]:
metadata_df = retrieve_nested_data_metadata()

Genres: 0it [00:00, ?it/s]

Belongs to Collection: 0it [00:00, ?it/s]

In [7]:
metadata_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,genre_id_5,genre_name_5,genre_id_6,genre_name_6,genre_id_7,genre_name_7,genre_id_8,genre_name_8,collection_id,part_of_collection
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,1,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10194.0,Toy Story Collection
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,2,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,3,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,119050.0,Grumpy Old Men Collection
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,4,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,5,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96871.0,Father of the Bride Collection


### Below here we just have some extremely basic EDA to see what the results look like

In [334]:
genre_cols = ['genre_id_1', 'genre_name_1',
                 'genre_id_2', 'genre_name_2', 
                 'genre_id_3', 'genre_name_3', 
                 'genre_id_4', 'genre_name_4', 
                 'genre_id_5', 'genre_name_5', 
                 'genre_id_6', 'genre_name_6', 
                 'genre_id_7', 'genre_name_7',
                 'genre_id_8', 'genre_name_8']
metadata_df[genre_cols].head(5)

,genre_id_1,genre_name_1,genre_id_2,genre_name_2,genre_id_3,genre_name_3,genre_id_4,genre_name_4,genre_id_5,genre_name_5,genre_id_6,genre_name_6,genre_id_7,genre_name_7,genre_id_8,genre_name_8
0,16.0,Animation,35.0,Comedy,10751.0,Family,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12.0,Adventure,14.0,Fantasy,10751.0,Family,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10749.0,Romance,35.0,Comedy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,35.0,Comedy,18.0,Drama,10749.0,Romance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,35.0,Comedy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### It seems as though it might be worth keeping just 1 & 2 since the rest of the columns have so many missing values. However I left the other columns in to give the option of use.

In [335]:
for col in genre_cols:
    print(f"{col} = {metadata_df[col].isna().sum()}")

genre_id_1 = 2433
genre_name_1 = 2433
genre_id_2 = 16974
genre_name_2 = 16974
genre_id_3 = 31430
genre_name_3 = 31430
genre_id_4 = 40993
genre_name_4 = 40993
genre_id_5 = 44366
genre_name_5 = 44366
genre_id_6 = 45199
genre_name_6 = 45199
genre_id_7 = 45356
genre_name_7 = 45356
genre_id_8 = 45380
genre_name_8 = 45380


In [336]:
collection_cols = ['collection_id', 'part_of_collection']
metadata_df[collection_cols].head(5)

,collection_id,part_of_collection
0,10194.0,Toy Story Collection
1,NaN,NaN
2,119050.0,Grumpy Old Men Collection
3,NaN,NaN
4,96871.0,Father of the Bride Collection


#### It looks like nearly half of the data entries for "belongs to collection" are NaN values. I would be intrigued to see wether or not leaving these in positively or negatively effects our various model's performance

In [337]:
for col in collection_cols:
    print(f"{col} = {metadata_df[col].isna().sum()}")

collection_id = 40897
part_of_collection = 40897


### Now, just dropping the original, un-expanded columns and saving to CSV for later use.

In [339]:
to_drop = ['belongs_to_collection', 'genres']
metadata_df = metadata_df.drop(columns = to_drop, axis = 1)

In [340]:
metadata_df.head()

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,genre_id_5,genre_name_5,genre_id_6,genre_name_6,genre_id_7,genre_name_7,genre_id_8,genre_name_8,collection_id,part_of_collection
0,False,30000000,http://toystory.disney.com/toy-story,1,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10194.0,Toy Story Collection
1,False,65000000,NaN,2,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,False,0,NaN,3,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.712900,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,119050.0,Grumpy Old Men Collection
3,False,16000000,NaN,4,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,False,0,NaN,5,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96871.0,Father of the Bride Collection


In [341]:
metadata_df.to_csv("metadata_expanded.csv", encoding = 'utf-8', ignore_index = True)

## Moving on to the Keywords CSV file!

In [22]:
import pandas as pd
import ast
from tqdm import tqdm

def retrieve_first_10_keywords():
    '''Function to retrieve the first 10 keywords for each movie in the Keywords file'''
    
    df = pd.read_csv("movies_keywords.csv")
    
    # Creating column names to account for the amount of sub-categories within each entry
    col_names = [f'keyword_id_{i}' for i in range(1, 11)]
    col_names += [f'keyword_name_{i}' for i in range(1, 11)]
    
    # Creating a list of dictionaries to store the keyword data
    keyword_list = []
    
    # Iterating through the dataframe with the iterrows method (its kinda like enumerate for DF's), 
    # using tqdm as a timer. Index from the "enumeration" is helpful for multi-indexing later.
    for index, row in tqdm(df.iterrows(), desc='keywords'):
        
        # Using ast library to evaluate a string formatted as a dictionary, which basically typecasts. 
        ## ast library works better than python's eval method IMO.
        keyword_data = ast.literal_eval(row[1])
        
        # Using enumerate here to be able to perform efficient multi-indexing.
        keywords_dict = {}
        for counter, dictionary in enumerate(keyword_data[:10], start=1):
            
            # Adding the "id" and "name" keys to a dictionary for each keyword
            keywords_dict[f'keyword_id_{counter}'] = dictionary.get('id')
            keywords_dict[f'keyword_name_{counter}'] = dictionary.get('name')
            
        # Appending the dictionary to the keyword list
        keyword_list.append(keywords_dict)
    
    # Creating a new dataframe from the keyword list
    keyword_df = pd.DataFrame(keyword_list, columns=col_names)
    
    # Concatenating the original dataframe with the keyword dataframe
    df = pd.concat([df[['id', 'title']], keyword_df], axis=1)
    
    # Returning the expanded dataframe
    return df


In [23]:
keyword_df = retrieve_nested_data_keywords()

keywords: 961it [00:00, 1519.56it/s]C:\Users\rache\AppData\Local\Temp\ipykernel_22656\1020484114.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[index, f'keyword_name_{counter}'] = value
C:\Users\rache\AppData\Local\Temp\ipykernel_22656\1020484114.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[index, f'keyword_id_{counter}'] = value
C:\Users\rache\AppData\Local\Temp\ipykernel_22656\1020484114.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` 

In [25]:
keyword_df.head()

,id,keywords,keyword_id_1,keyword_name_1,keyword_id_2,keyword_name_2,keyword_id_3,keyword_name_3,keyword_id_4,keyword_name_4,...,keyword_id_145,keyword_name_145,keyword_id_146,keyword_name_146,keyword_id_147,keyword_name_147,keyword_id_148,keyword_name_148,keyword_id_149,keyword_name_149
0,1,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",931.0,jealousy,4290.0,toy,5202.0,boy,6054.0,friendship,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,"[{'id': 10090, 'name': 'board game'}, {'id': 1...",10090.0,board game,10941.0,disappearance,15101.0,based on children's book,33467.0,new home,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392...",1495.0,fishing,12392.0,best friend,179431.0,duringcreditsstinger,208510.0,old men,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,"[{'id': 818, 'name': 'based on novel'}, {'id':...",818.0,based on novel,10131.0,interracial relationship,14768.0,single mother,15160.0,divorce,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...",1009.0,baby,1599.0,midlife crisis,2246.0,confidence,4995.0,aging,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### ^^ Got some fragmentation warnings when running the above code. ^^ However, the end result is just fine. I would like to understand why, but its beyond my scope of understanding.

#### The Warning Itself:
PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[index, f'keyword_name_{counter}'] = value

In [355]:
keyword_col_names = ['keyword_id_1', 'keyword_name_1',
                     'keyword_id_2', 'keyword_name_2', 
                     'keyword_id_3', 'keyword_name_3', 
                     'keyword_id_4', 'keyword_name_4', 
                     'keyword_id_5', 'keyword_name_5', 
                     'keyword_id_6', 'keyword_name_6', 
                     'keyword_id_7', 'keyword_name_7',
                     'keyword_id_8', 'keyword_name_8',
                     'keyword_id_9', 'keyword_name_9',
                     'keyword_id_10', 'keyword_name_10']
keyword_df[keyword_col_names].head(5)

,keyword_id_1,keyword_name_1,keyword_id_2,keyword_name_2,keyword_id_3,keyword_name_3,keyword_id_4,keyword_name_4,keyword_id_5,keyword_name_5,keyword_id_6,keyword_name_6,keyword_id_7,keyword_name_7,keyword_id_8,keyword_name_8,keyword_id_9,keyword_name_9,keyword_id_10,keyword_name_10
0,931.0,jealousy,4290.0,toy,5202.0,boy,6054.0,friendship,9713.0,friends,9823.0,rivalry,165503.0,boy next door,170722.0,new toy,187065.0,toy comes to life,NaN,NaN
1,10090.0,board game,10941.0,disappearance,15101.0,based on children's book,33467.0,new home,158086.0,recluse,158091.0,giant insect,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1495.0,fishing,12392.0,best friend,179431.0,duringcreditsstinger,208510.0,old men,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,818.0,based on novel,10131.0,interracial relationship,14768.0,single mother,15160.0,divorce,33455.0,chick flick,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1009.0,baby,1599.0,midlife crisis,2246.0,confidence,4995.0,aging,5600.0,daughter,10707.0,mother daughter relationship,13149.0,pregnancy,33358.0,contraception,170521.0,gynecologist,NaN,NaN


### Again it looks like the Keywords have a lot of missing values overall. However, hopefully, it is still useful data for our future modelling.

In [356]:
for col in keyword_col_names:
    print(f"{col} = {keyword_df[col].isna().sum()}")

keyword_id_1 = 14750
keyword_name_1 = 14750
keyword_id_2 = 21359
keyword_name_2 = 21359
keyword_id_3 = 26170
keyword_name_3 = 26170
keyword_id_4 = 30441
keyword_name_4 = 30441
keyword_id_5 = 33738
keyword_name_5 = 33738
keyword_id_6 = 36595
keyword_name_6 = 36595
keyword_id_7 = 38565
keyword_name_7 = 38565
keyword_id_8 = 40061
keyword_name_8 = 40061
keyword_id_9 = 41112
keyword_name_9 = 41112
keyword_id_10 = 41973
keyword_name_10 = 41973


### Now, just dropping the original, un-expanded columns and saving to CSV for later use.

In [357]:
keyword_df = keyword_df.drop(columns = 'keywords', index = 1)

In [359]:
keyword_df.to_csv('keywords_expanded.csv', encoding = 'utf-8', ignore_index = True)

# And that's it for Rachel's strong-arm method for extracting the nested data! 